# Modos normales de vibración

Nota: a este documento lo podremos ejecutar de manera dinámica si tienen instalado:

- Python 3.5 o más nuevo instalado.
- [Jupyter Notebook](https://jupyter.readthedocs.io/en/latest/install.html).
- [FEniCS](https://fenicsproject.org/).

La visualización del mismo es óptima utilizando Jupyter Notebook.

### Referencias
- Capítulo 11 de M. G. Larson y F. Bengzon, The FEM: theory, implementations and applications, Springer, (2010).

- Capítulo 20 sección 5 de O. C. Zienkiewicz, El método de los elementos finitos, Editorial Reverté, (2018). Se puede acceder por la plataforma eLibro de UTN.

- Sacamos parte del código del sitio de J. Bleyer, [seguir enlace](https://comet-fenics.readthedocs.io/en/latest/).

- Las herramientas importantes que se usan para la resolución de matrices grandes y sparse son: [SLEPc](https://slepc.upv.es/documentation/slepc.pdf) y [PETSc](https://www.mcs.anl.gov/petsc/). Están implementadas en FEniCS.

## Introducción

En este tutorial comentaremos lo que se denomina Análisis Modal. La idea general es estudiar el comportamiento de una estructura en una vibración mecánica de una frecuencia fija. Es decir, encontraremos las frecuencias de oscilación naturales de la estructura. Este análisis se debe hacer en el diseño de piezas para identificar frecuencias de resonancia (por definición, autovalores). Esto es importante porque, como lo vimos en los cursos de mecánica inicial, si la estructura es sometida a una oscilación forzada mediante una excitación externa con una frecuencia similar puede llevar a una resonancia y desgastarla (incluso destruirla). 

Recordemos cuando estudiamos matrices y calculamos autovalores y autovectores, el procedimiento es el siguiente. Supongamos que tenemos una matriz $K$ de $n\times n$, un vector $\mathbf{z}$ de $n\times 1$. Si le pedimos a la matriz $K$ que sea semidefinida positiva (o definida positiva) entonces existen $n$ autovalores ($\lambda_{i}$) y sus correspondientes $n$ autovectores ($z_{i}$) que satisfacen la siguiente:

$$K \mathbf{z} = \lambda \mathbf{z} \tag{1}$$

las soluciones vienen de a pares $\left(\lambda_{i},z_{i}\right)$. A la solución de este problema se lo denomina **Problema de valores propios estándar**. Otro problema más interesante para nosotros es el **Problema de valores propios generalizados** que nos permite tener información de la vibración dinámica libre de la pieza. A este problema lo podemos escribir como $K \mathbf{z} = \lambda M \mathbf{z}$ o lo que es equivalente:

$$K \mathbf{z} - \lambda M \mathbf{z}=0 \tag{2}$$

donde tanto K como M son matrices (de rigidez y de masa). Nuevamente aquí debemos encontrar $\mathbf{z}$ y $\lambda$. 

A continuación mostraremos un razonamiento que nos permitirá entender por qué hablamos de vibración dinámica libre y luego, realizaremos el cómputo con el método de elementos finitos.

### Ecuación de balance de momento

La formulación dura será la misma, con la diferencia que se asume $b=0$, es decir, el peso distribuido en toda la pieza no lo consideramos (aunque la masa "inercial" si, porque estamos interesados en una oscilación en torno a un punto, el problema dinámico):

$$ \left \{ \begin{array}{l} \rho \ddot{u}-\nabla\cdot \sigma = 0 \ \  \text{en} \ \ \Omega\times \text{I} \\ u=g\text{ en } \ \partial \Omega_{D}\times \text{I} \text{ (condición de borde Dirichlet) } 
\end{array} \tag{3}\right .$$

Como relación constitutiva entre $\sigma$ y $\varepsilon$, consideramos la siguiente:

$$\sigma = 2\mu\varepsilon+\left(\lambda tr\left(\varepsilon\right)\right)I.$$

Supongamos que en la Ec. (3) se reemplaza el desplazamiento por la función $u = z sen \left(\omega t\right)$. Es decir, que consideramos desplazamientos oscilatorios de frecuencia $\omega$ (esto se le dice solución de onda plana). Aquí consideramos que $z$ no depende del tiempo, pero si del espacio. La Ec. (3) queda:

$$ \left \{ \begin{array}{l} -\rho \omega^{2}zsen\left(\omega t\right)-\nabla\cdot \sigma(z sen\left(\omega t\right)) = 0 \ \  \text{en} \ \ \Omega\times \text{I} \\ u=g\text{ en } \ \partial \Omega_{D}\times \text{I} \text{ (condición de borde Dirichlet) } 
\end{array} \tag{4}\right .$$

Esta ecuación se puede simplificar porque es lineal con el $sen\left(\omega t\right)$, y al estar en ambas partes de la misma se puede cancelar, por lo tanto:

$$-\rho \omega^{2}z-\nabla\cdot \sigma(z) = 0\tag{5}$$

Una vez discretizado el espacio la Ec. (5) se puede pensar como:

$$\left( -\omega^{2}M+K\right) \mathbf{z}=0\tag{6}$$

comparando con la Ec. (2), aquí hemos reemplazado $\lambda = \omega^{2}$.

### Formulación variacional

Para calcular la formulación variacional tomamos la Ec. (5) y multiplicamos por la función de prueba $v$ e integramos en el dominio $\Omega$. Si aplicamos la fórmula de Green queda:
 
$$\int_{\Omega} -\rho \omega^{2}z \ v \ dx+\int_{\Omega} \sigma \left(z\right) : \varepsilon\left(v\right) \ dx = 0 \tag{7}.$$

Con esta expresión ya podemos discretizar y llegar a la Ec. (6). Noten que la parte izquierda tiene que ver con la masa (aceleración) $M$ y la parte derecha tiene que ver con la rigidez $K$. 

El ejemplo que mostraremos a continuación es la barra empotrada en voladizo con la que hemos estado trabajando. Comentaremos a continuación el código donde está implementado.


### Código

Todo el código siguiente está implementado en el *ejemplo18.py* subido a la carpeta de ejemplos. El comienzo es similar a lo que veníamos trabajando, no lo repetiremos aquí. 

In [1]:
from dolfin import *
import numpy as np

L, B, H = 20., 0.5, 1.
Nx = 200
Ny = int(B/L*Nx)+1
Nz = int(H/L*Nx)+1

mesh = BoxMesh(Point(0.,0.,0.),Point(L,B,H), Nx, Ny, Nz)

E, nu = Constant(1e5), Constant(0.)
rho = Constant(1e-3)
mu = E/2./(1+nu)
lmbda = E*nu/(1+nu)/(1-2*nu)

def eps(v):
    return sym(grad(v))
def sigma(v):
    dim = v.geometric_dimension()
    return 2.0*mu*eps(v) + lmbda*tr(eps(v))*Identity(dim)

V = VectorFunctionSpace(mesh, 'Lagrange', degree=1)
z_ = TrialFunction(V)
v = TestFunction(V)

def izquierda(x, on_boundary):
    return near(x[0],0.)

bc = DirichletBC(V, Constant((0.,0.,0.)), izquierda)

Construiremos las matrices $M$ y $K$ y utilizaremos solvers específicos para acelerar este tipo de problemas. En general, para matrices grandes y ralas (como lo son las matrices con las que trabajamos aquí) existen herramientas bien desarrolladas de código libre que no pertenecen a FEniCS pero que sin embargo se pueden utilizar desde FEniCS, y este último tiene herramientas para escribir y acceder a aquellas de manera simple. La herramienta más conocida es [PETSc](https://www.mcs.anl.gov/petsc/) (Portable, Extensible Toolkit for Scientific Computation) y una extensión para valores propios [SLEPc](https://slepc.upv.es/documentation/slepc.pdf) (Scalable Library for Eigenvalue Problem Computations).

Utilizaremos nuevamente la función *assemble_system* y *assemble* para encontrar las matrices $K$ y $M$ (con el sistema discretizado en los elementos definidos por la malla). Utilizamos la Ec. (7) para definir las formulaciones variacionales, y parte bilineal y lineal.

Comenzaremos por la rigidez $K$, para ello utilizamos la parte derecha de la Ec. (7) y la escribimos directamente:

In [2]:
k_form = inner(sigma(v),eps(z_))*dx
l_form = Constant(1.)*z_[0]*dx

Hemos creado también una función *l_form* se le dice "tipo dummy", que no tiene sentido físico sino que la utilizaremos para obtener la matriz $K$. Luego le decimos a FEniCS que utilizaremos matrices para las herramientas PETSc, por eso las creamos:

In [3]:
K = PETScMatrix()
b = PETScVector()
assemble_system(k_form, l_form, bc, A_tensor=K, b_tensor=b)

(<dolfin.cpp.la.PETScMatrix at 0x7f3db4b01e08>,
 <dolfin.cpp.la.PETScVector at 0x7f3db4b01e60>)

Aquí ya aplicamos la función *assemble_system* y guardamos en $K$ la matriz de rigidez, noten que a $b$ no lo vamos a utilizar para nada.

Para encontrar la matriz $M$, utilizamos la parte izquierda de la Ec. (7) y procedemos como sigue:

In [4]:
m_form = rho*dot(v,z_)*dx
M = PETScMatrix()
assemble(m_form, tensor=M)

Luego, preparamos el solver de SLEPc, específico para obtener valores principales y lo único que tenemos que pasarle son las matrices $K$ y $M$. Con el parámetro *gen_hermitian* la rutina *SLEPcEigenSolver* ya "entiende" que tiene que resolver un problema de valores propios generalizados con matrices Hermíticas. El método *shift-and-invert* es un método que se utiliza para acelerar y resolver este tipo de problemas (se puede ver la referencia [SLEPc](https://slepc.upv.es/documentation/slepc.pdf) para una idea del método). 

In [5]:
eigensolver = SLEPcEigenSolver(K, M)
eigensolver.parameters['problem_type'] = 'gen_hermitian'
eigensolver.parameters['spectral_transform'] = 'shift-and-invert'
eigensolver.parameters['spectral_shift'] = 0.

Ahora le pedimos a SLEPc que compute los 6 primeros valores principales llamando la función *solve* y extrayendo los correspondientes autovalores y autovectores.

In [7]:
N_eig = 6
print("Computando los primeros {} valores propios...".format(N_eig))
eigensolver.solve(N_eig)
from scipy.optimize import root
from math import cos, cosh

falpha = lambda x: cos(x)*cosh(x)+1
alpha = lambda n: root(falpha, (2*n+1)*pi/2.)['x'][0]

for i in range(N_eig):
    # Extrae los autovalores y autovectores
    r, c, rx, cx = eigensolver.get_eigenpair(i)

    print('autovalores real e imaginarios: ',r,'+j*',c)
    # Frecuencia calculada con FEM
    freq_3D = sqrt(r)/2/pi

    # Teórica aproximada
    if i % 2 == 0: # solución con eje débil
        I_bend = H*B**3/12.
    else:          # solución con eje fuerte
        I_bend = B*H**3/12.
    rho1 = 1e-3
    E1 = 1e5
    freq_beam = alpha(i/2)**2*sqrt(E1*I_bend/(rho1*B*H*L**4))/2/pi
    
    print("Con FEM: {0:8.5f} [Hz]   Teoría: {1:8.5f} [Hz]".format(freq_3D, freq_beam))

    # Inicializo una función que viva en V para guardar los autovalores
    eigenmode = Function(V,name="Eigenvector "+str(i))
    eigenmode.vector()[:] = rx


Computando los primeros 6 valores propios...
autovalores real e imaginarios:  179.265219896501 +j* 0.0
Con FEM:  2.13092 [Hz]   Teoría:  2.01925 [Hz]
autovalores real e imaginarios:  661.9928567749275 +j* 0.0
Con FEM:  4.09493 [Hz]   Teoría:  4.03850 [Hz]
autovalores real e imaginarios:  7005.429265316417 +j* 0.0
Con FEM: 13.32102 [Hz]   Teoría: 12.65443 [Hz]
autovalores real e imaginarios:  25498.773962844734 +j* 0.0
Con FEM: 25.41440 [Hz]   Teoría: 25.30886 [Hz]
autovalores real e imaginarios:  54489.068630294 +j* 0.0
Con FEM: 37.15137 [Hz]   Teoría: 35.43277 [Hz]
autovalores real e imaginarios:  193313.33070703907 +j* 0.0
Con FEM: 69.97631 [Hz]   Teoría: 70.86554 [Hz]


Noten que la función *eigensolver.get_eigenpair* tiene 4 salidas r, c, rx, y cx las dos primeras son la parte real e imaginaria del autovalor y las dos últimas son los autovalores reales e imaginarios (nosotros tendremos sólo parte real en este caso). 

La siguiente [animación](https://comet-fenics.readthedocs.io/en/latest/_images/vibration_modes.gif) muestra los primeros dos modos en sentido del eje débil y fuerte.